Ordered Time = Difference 1 hour upto 72 hours (3 days)<br>
Run Time = 00, 06, 12, 18<br>
<br>
Source Field id (Range 91 to  135)<br>

<table>
<tr>
<th>HRRR</th>
<th>NAM</th>
<th>GFS</th>
<th>Parameter</th>
<th>Units</th>
</tr>

<tr>
<td>91</td>
<td>106</td>
<td>121</td>
<td>2m Temperature</td>
<td>K</td>
</tr>

<tr>
<td>92</td>
<td>107</td>
<td>122</td>
<td>Visibility</td>
<td>m</td>
</tr>
    
<tr>
<td>93</td>
<td>108</td>
<td>123</td>
<td>Rain</td>
<td>no unit</td>
</tr>  

<tr>
<td>94</td>
<td>109</td>
<td>124</td>
<td>Ice</td>
<td>no unit</td>
</tr>
    
<tr>
<td>95</td>
<td>110</td>
<td>125</td>
<td>Freezing Rain</td>
<td>no unit</td>
</tr>

<tr>
<td>96</td>
<td>111</td>
<td>126</td>
<td>Snow</td>
<td>no unit</td>
</tr>
 
<tr>
<td>97</td>
<td>112</td>
<td>127</td>
<td>Composite Reflectivity</td>
<td>dbZ</td>
</tr>
    
<tr>
<td>98</td>
<td>113</td>
<td>128</td>
<td>2m Humidity</td>
<td>kg/kg</td>
</tr>

<tr>
<td>99</td>
<td>114</td>
<td>129</td>
<td>Surface Pressure</td>
<td>Pa</td>
</tr>

<tr>
<td>100</td>
<td>115</td>
<td>130</td>
<td>10m Wind U-component</td>
<td>m/s</td>
</tr>
    
<tr>
<td>101</td>
<td>116</td>
<td>131</td>
<td>10m Wind V-component</td>
<td>deg</td>
</tr>
    
<tr>
<td>102</td>
<td>117</td>
<td>132</td>
<td>10m Wind Speed</td>
<td>m/s</td>
</tr>
    
<tr>
<td>103</td>
<td>118</td>
<td>133</td>
<td>10m Wind Direction</td>
<td>deg</td>
</tr>

<tr>
<td>104</td>
<td>119</td>
<td>134</td>
<td>Gust Speed</td>
<td>m/s</td>
</tr>
    
<tr>
<td>105</td>
<td>120</td>
<td>135</td>
<td>Cloud Cover</td>
<td>%</td>
</tr>
</table>

In [1]:
# import libraries
import json
import pandas as pd
import csv
import requests
import time
import plotly.express as px

In [3]:
# Getting top 20 citites list
top20cities = []
with open(r'C:\Users\Dhivya\Desktop\Git\Projects\vortexweather\input\worldcities.csv',encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile)
    for i,row in enumerate(reader):
        top20cities.append({"city" : row['city_ascii'], "country" : row['country'], "lat" : row['lat'], "lng" : row['lng'] , "population" : row["population"]})
        if i == 19:
            break

In [ ]:
# Getting the json dump of each city and storing it with city.json
for i in range(20):
    lat = top20cities[i]['lat']
    long = top20cities[i]['lng']
    city = top20cities[i]['city']
    startTime = round(time.time())
    endTime = round(time.time()) + 3 * 24 * 60 * 60
    payload = {'lat': lat, 'lon': long, 'start' : startTime, 'end' : endTime}
    page = requests.get('https://api.vortexweather.tech/api/wx',params = payload)
    data = page.json()
    with open(r'C:\Users\Dhivya\Desktop\Git\Projects\vortexweather\output\{}.json'.format(city), 'w') as outfile:
        json.dump(data, outfile)

In [4]:
# load the file
def loadjsonfile(filename):
    file = open(filename)
    data = json.load(file) 
    return data

In [5]:
# Get GFS 2m Temperature Data
def getGFS2mTemperatureData(data):
    gfs_run_time = []
    gfs_2mTemperature_K = []
    gfs_orderTime = []
    for i in data["ordered_times"]:
        for item in data["data"][str(i)]:
            if(item['src_field_id'] == 121):
                gfs_orderTime.append(str(i))
                gfs_run_time.append(item['run_time'])
                gfs_2mTemperature_K.append(item['value'])
    gfs_2mTemperature_F = [((x - 273.15) * 9/5 + 32) for x in gfs_2mTemperature_K]
    gfs_2mTemperature_Data = pd.DataFrame({"GFS Run Time":gfs_run_time, "GFS Order Time" : gfs_orderTime, "GFS 2m Temperature Value" : gfs_2mTemperature_F})
    fig = px.line(gfs_2mTemperature_Data, x='GFS Order Time', y='GFS 2m Temperature Value', color= 'GFS Run Time', title= "Temperature Data")
    fig.update_traces(mode="markers+lines")
    fig.show()
    return gfs_2mTemperature_Data

In [6]:
# Get NAM 2m Temperature Data
def getNAM2mTemperatureData(data):
    nam_run_time = []
    nam_2mTemperature_K = []
    nam_orderTime = []
    for i in data["ordered_times"]:
        for item in data["data"][str(i)]:
            if(item['src_field_id'] == 106):
                nam_orderTime.append(str(i))
                nam_run_time.append(item['run_time'])
                nam_2mTemperature_K.append(item['value'])
    nam_2mTemperature_F = [((x - 273.15) * 9/5 + 32) for x in nam_2mTemperature_K]
    nam_2mTemperature_Data = pd.DataFrame({"NAM Run Time" : nam_run_time, "NAM Order Time" : nam_orderTime, "NAM 2m Temperature Value" : nam_2mTemperature_F})
    fig = px.line(nam_2mTemperature_Data, x='NAM Order Time', y='NAM 2m Temperature Value', color= 'NAM Run Time', title= "Temperature Data")
    fig.update_traces(mode="markers+lines")
    fig.show()
    return nam_2mTemperature_Data

In [17]:
# Get GFS Rain Data
def getGFSRainData(data):
    gfs_run_time = []
    gfs_rain = []
    gfs_orderTime = []
    for i in data["ordered_times"]:
        for item in data["data"][str(i)]:
            if(item['src_field_id'] == 123):
                gfs_orderTime.append(str(i))
                gfs_run_time.append(item['run_time'])
                gfs_rain.append(item['value'])
    gfs_rain_Data = pd.DataFrame({"GFS Run Time":gfs_run_time, "GFS Order Time" : gfs_orderTime, "GFS Rain" : gfs_rain})
    fig = px.line(gfs_rain_Data, x='GFS Order Time', y='GFS Rain', color= 'GFS Run Time', title= "Rain Data")
    fig.update_traces(mode="markers+lines")
    fig.show()
    fig.write_image('fig1.png')
    return gfs_rain_Data

In [18]:
# Getting values for all the 20 cities
for i in range(5):
    city = top20cities[i]["city"]
    print(city)
    cityname = loadjsonfile(r'C:\Users\Dhivya\Desktop\Git\Projects\vortexweather\output\{}.json'.format(city))
    result = getGFS2mTemperatureData(cityname)
    result.to_csv(r'C:\Users\Dhivya\Desktop\Git\Projects\vortexweather\output\{}.csv'.format(city))
    print(result)
    print("\n")
    print("\n")

Tokyo


     GFS Run Time GFS Order Time  GFS 2m Temperature Value
0      1593669600     1593712800                 78.286433
1      1593648000     1593712800                 78.593611
2      1593453600     1593712800                 79.511189
3      1593410400     1593712800                 81.059767
4      1593583200     1593712800                 79.144740
..            ...            ...                       ...
977    1593604800     1593968400                 76.910011
978    1593561600     1593968400                 77.629945
979    1593648000     1593968400                 74.900941
980    1593540000     1593968400                 78.530000
981    1593583200     1593968400                 77.856813

[982 rows x 3 columns]




New York


     GFS Run Time GFS Order Time  GFS 2m Temperature Value
0      1593669600     1593712800                 90.526411
1      1593648000     1593712800                 92.093611
2      1593453600     1593712800                 90.671156
3      1593410400     1593712800                 89.879756
4      1593583200     1593712800                 90.844740
..            ...            ...                       ...
977    1593604800     1593968400                 85.730000
978    1593561600     1593968400                 79.249989
979    1593648000     1593968400                 85.160963
980    1593540000     1593968400                 77.090022
981    1593583200     1593968400                 80.916835

[982 rows x 3 columns]




Mexico City


     GFS Run Time GFS Order Time  GFS 2m Temperature Value
0      1593669600     1593712800                 72.526411
1      1593648000     1593712800                 71.933589
2      1593453600     1593712800                 71.411189
3      1593410400     1593712800                 71.339778
4      1593583200     1593712800                 70.864729
..            ...            ...                       ...
977    1593604800     1593968400                 70.430000
978    1593561600     1593968400                 70.609956
979    1593648000     1593968400                 70.400941
980    1593540000     1593968400                 70.790022
981    1593583200     1593968400                 70.116835

[982 rows x 3 columns]




Mumbai


     GFS Run Time GFS Order Time  GFS 2m Temperature Value
0      1593669600     1593712800                 84.766444
1      1593648000     1593712800                 84.533589
2      1593453600     1593712800                 84.371156
3      1593410400     1593712800                 83.939778
4      1593583200     1593712800                 84.364729
..            ...            ...                       ...
977    1593604800     1593968400                 83.930000
978    1593561600     1593968400                 83.569978
979    1593648000     1593968400                 83.900941
980    1593540000     1593968400                 83.570033
981    1593583200     1593968400                 84.516835

[982 rows x 3 columns]




Sao Paulo


     GFS Run Time GFS Order Time  GFS 2m Temperature Value
0      1593669600     1593712800                 67.666444
1      1593648000     1593712800                 67.973622
2      1593453600     1593712800                 68.531178
3      1593410400     1593712800                 65.939778
4      1593583200     1593712800                 68.704707
..            ...            ...                       ...
977    1593604800     1593968400                 78.170033
978    1593561600     1593968400                 78.169978
979    1593648000     1593968400                 71.660963
980    1593540000     1593968400                 77.990022
981    1593583200     1593968400                 78.936824

[982 rows x 3 columns]






In [41]:
cityname = loadjsonfile(r"C:\Users\Dhivya\New York.json")
result = getNAM2mTemperatureData(cityname)
result

,NAM Run Time,NAM Order Time,NAM 2m Temperature Value
0,1593626400,1593712800,87.421949
1,1593691200,1593712800,89.347963
2,1593561600,1593712800,88.465980
3,1593604800,1593712800,87.656013
4,1593669600,1593712800,89.761982
...,...,...,...
275,1593691200,1593892800,80.311982
276,1593691200,1593896400,80.635969
277,1593691200,1593900000,78.583998
278,1593691200,1593903600,74.930000


In [45]:
cityname = loadjsonfile(r"C:\Users\Dhivya\New York.json")
result = getGFSRainData(cityname)
result

,GFS Run Time,GFS Order Time,GFS Rain
0,1593669600,1593712800,0.0
1,1593648000,1593712800,0.0
2,1593496800,1593712800,0.0
3,1593583200,1593712800,0.0
4,1593475200,1593712800,0.0
...,...,...,...
977,1593604800,1593968400,0.0
978,1593561600,1593968400,0.0
979,1593648000,1593968400,0.0
980,1593540000,1593968400,0.0
